# Importing an `snnTorch` SNN model to LAVA

In [1]:
# Show current directory
import os
curr_dir = os.getcwd()
print(curr_dir)

# Check if the current WD is the file location
if "/thesis-lava/src/nir" not in os.getcwd():
    # Set working directory to this file location
    file_location = f"{os.getcwd()}/thesis-lava/src/nir"
    print("File Location: ", file_location)

    # Change the current working Directory
    os.chdir(file_location)

    # New Working Directory
    print("New Working Directory: ", os.getcwd())

/home/monkin/Desktop/feup/thesis
File Location:  /home/monkin/Desktop/feup/thesis/thesis-lava/src/nir
New Working Directory:  /home/monkin/Desktop/feup/thesis/thesis-lava/src/nir


In [2]:
import numpy as np
import nir
import matplotlib.pyplot as plt

nir_network = nir.read("nir_model_cuba.nir")

In [3]:
# Print the network summary
print(f"NIR Network Info: Nº Nodes: {len(nir_network.nodes)} | Nº Edges: {len(nir_network.edges)}")

print(nir_network.nodes["2"])
print(nir_network.edges)


NIR Network Info: Nº Nodes: 6 | Nº Edges: 5
Linear(weight=array([[-0.0161455 ,  0.01705358,  0.01007112, ...,  0.00985886,
         0.03937952, -0.03549746],
       [ 0.03603304, -0.03541515,  0.01668479, ...,  0.04050541,
         0.0300939 ,  0.03487029],
       [ 0.03381281, -0.02337706,  0.02692366, ...,  0.02496983,
         0.03662976, -0.03068166],
       ...,
       [ 0.00337175,  0.01589908, -0.02817665, ...,  0.01133711,
        -0.01727862,  0.04292991],
       [-0.00492689,  0.03207321,  0.01319447, ..., -0.03240683,
         0.00964233, -0.03846167],
       [-0.01455489,  0.01560639, -0.02629465, ...,  0.03446307,
        -0.03139296,  0.00727098]], shape=(10, 500), dtype=float32))
[('3', 'output'), ('input', '0'), ('2', '3'), ('1', '2'), ('0', '1')]


### How to fetch the number of neurons in each `NIR.CubaLIF` layer

In [4]:
nirLIF1 = nir_network.nodes["1"]
print(nirLIF1)

CubaLIF(tau_syn=np.float32(0.00014285714), tau_mem=np.float32(0.00012499999), r=np.float32(1.25), v_leak=np.float32(0.0), v_threshold=np.float32(1.0), w_in=np.float32(1.4285715), input_type={'input': array([], dtype=float64)}, output_type={'output': array([], dtype=float64)}, metadata={}, num_neurons=np.int64(500))


In [5]:

MANUAL_CREATION = True

if MANUAL_CREATION:
    # TODO: This is not the original code. I changed it because it had errors..
    """ ng = nir.NIRGraph(
        nodes={
            "input": nir.Input(input_type=np.array([3])),
            # "affine": nir.Affine(weight=np.array([[8, 2, 10], [14, 3, 14]]).T, bias=np.array([1, 2])),
            "lif": nir.CubaLIF(
                # tau=np.array([1] * 2),
                r=np.array([1 / 1e-4] * 2),
                tau_syn=np.array([1] * 2),
                tau_mem=np.array([1] * 2),
                v_leak=np.array([0] * 2),
                v_threshold=np.array([1] * 2),
            ),
            "out": nir.Output(np.array([3])),
        },
        edges=[
            # ("input", "affine"),
            # ("affine", "lif"),
            ("input", "lif"),
            ("lif", "out")
        ],
    ) """
    ng = nir.NIRGraph(
        nodes={
            "input": nir.Input(input_type=np.array([3])),
            "affine": nir.Affine(weight=np.array(
                [[8, 2, 10], [14, 3, 14]]).T, bias=np.array([0, 0])),
            "lif": nir.LIF(
                tau=np.array([1] * 2),
                r=np.array([1] * 2),
                v_leak=np.array([0] * 2),
                v_threshold=np.array([1] * 2),
            ),
            "output": nir.Output(np.array([3])),
        },
        edges=[(0, 1), (1, 2), (2, 3)],
    )

In [6]:
from nir_to_lava_edit import ImportConfig, LavaLibrary, import_from_nir
# from nir_to_lava import ImportConfig, LavaLibrary, import_from_nir

config = ImportConfig(
    dt=1e-4, fixed_pt=False, on_chip=False, library_preference=LavaLibrary.Lava,
)

In [10]:
nir2lava_network, startNodes, endNodes = import_from_nir(
    nir_network,  # ng, # 
    config
)

In [12]:
print(nir2lava_network)
print(startNodes)
print(endNodes)

{'0': <lava.proc.dense.process.Dense object at 0x7facbed1bc70>, '1': [<lava.proc.dense.process.Dense object at 0x7facbeda6170>, <lava.proc.lif.process.LIF object at 0x7facbeebbca0>], '2': <lava.proc.dense.process.Dense object at 0x7facbed44df0>, '3': [<lava.proc.dense.process.Dense object at 0x7facbeed0eb0>, <lava.proc.lif.process.LIF object at 0x7facbed5cf40>]}
['0']
['3']


I was able to make it run without errors! However, it seems like the code to convert a NIR network to a LAVA network is not complete.

## How to Use the Lava Processes imported with NIR
It seems like `import_from_nir` returns a map containing the Lava Processes that were imported. Let's try to use them

### Print the ports of the Network Processes

In [21]:
from lava.magma.core.process.process import AbstractProcess

# Print the Network Ports
for key, proc in nir2lava_network.items():
    print(f"Key: {key} | Proc: {proc}")

    if isinstance(proc, list):
        # Network Node contains multiple Processes
        # Note: Max Depth is 2
        for inner_key,inner_proc in enumerate(proc):
            print(f"Key: ({key},{inner_key}) | Proc: {inner_proc}")
            # Print the Network Ports
            for port in inner_proc.in_ports:
                print(f"Proc: {inner_proc.name:<5} Port Name: {port.name:<5} Size: {port.size}")
            for port in inner_proc.out_ports:
                print(f"Proc: {inner_proc.name:<5} Port Name: {port.name:<5} Size: {port.size}")
    else:
        # Print the Network Ports
        for port in proc.in_ports:
            print(f"Proc: {proc.name:<5} Port Name: {port.name:<5} Size: {port.size}")
        for port in proc.out_ports:
            print(f"Proc: {proc.name:<5} Port Name: {port.name:<5} Size: {port.size}")

Key: 0 | Proc: <lava.proc.dense.process.Dense object at 0x7facbed1bc70>
Proc: Process_6 Port Name: s_in  Size: 784
Proc: Process_6 Port Name: a_out Size: 500
Key: 1 | Proc: [<lava.proc.dense.process.Dense object at 0x7facbeda6170>, <lava.proc.lif.process.LIF object at 0x7facbeebbca0>]
Key: (1,0) | Proc: <lava.proc.dense.process.Dense object at 0x7facbeda6170>
Proc: Process_8 Port Name: s_in  Size: 500
Proc: Process_8 Port Name: a_out Size: 500
Key: (1,1) | Proc: <lava.proc.lif.process.LIF object at 0x7facbeebbca0>
Proc: cuba lif Port Name: a_in  Size: 500
Proc: cuba lif Port Name: s_out Size: 500
Key: 2 | Proc: <lava.proc.dense.process.Dense object at 0x7facbed44df0>
Proc: Process_9 Port Name: s_in  Size: 500
Proc: Process_9 Port Name: a_out Size: 10
Key: 3 | Proc: [<lava.proc.dense.process.Dense object at 0x7facbeed0eb0>, <lava.proc.lif.process.LIF object at 0x7facbed5cf40>]
Key: (3,0) | Proc: <lava.proc.dense.process.Dense object at 0x7facbeed0eb0>
Proc: Process_11 Port Name: s_in  S

# Notes
- To import an `snn-torch` model using `NIR`, I had to modify `nir_to_lava`, `snntorch`, and also `nirtorch` source code.  